# Paper implementation challenge - Quantum Compression Algorithm for Symmetric States

This notebook is part of the paper implementation challenge, where we implement the algorithm described in the paper [[Efficient compression of quantum information](https://arxiv.org/abs/0907.1764)] by Martin Plesch and Vladimír Bužek. The paper presents a method for efficiently constructing a quantum circuit that compresses multiple copies of identical quantum states into a lower-dimensional Hilbert space. This compression can be generalized not only to identical states but to any quantum state that is symmetric under permutation of the qubits.

## Mathematical formulation
Any identical quantum state can be written as $\left|\psi\right\rangle^{\otimes N} = \underset{k=0}{\overset{N}{\sum}}\alpha^{N-k}\beta^{k}\sqrt{\binom{N}{k}}\left|N;k\right\rangle
     = A_k\left|N;k\right\rangle$, where the $\left|N;k\right\rangle = \sqrt{\binom{N}{k}}^{-1} \sum_{\sigma} \sigma \left|1\right\rangle^{\otimes k} \otimes \left|0\right\rangle^{\otimes (N-k)}$. 

The circuit presented in the original paper transforms each $\left|N;k\right\rangle$ into a new state $\left|C_k\right\rangle$, which corresponds to a state having only 1 excitation $\left(\left|C_k\right\rangle = \left|0\right\rangle^{\otimes{k-1}}\otimes \left|1\right\rangle \otimes \left|0\right\rangle^{\otimes(N-k)}\right)$. So, the goal is to construct a circuit that will perform this kind of operation, i.e. $U(\left|N;k\right\rangle) = \left|C_k\right\rangle$.

The final step of the algorithm will be to transform each of the state $\left|C_k\right\rangle$ into a new state so that in total just a number of $\log_2(N+1)$ qubits will be used. 

The operation is reversible, so that the decompression could be performed.


## Solving with the Classiq Platform

In [2]:
from classiq import *
from classiq.qmod.symbolic import pi
from scipy.special import comb
import numpy as np
from math import sqrt

### Defining the classical functions that are necessary for constructing the U gate

#### The first part of the $U$ : $U_{ab}$
We are defining the function that is creating the gate U_ab. This is part of the gate U from the paper and represents
a three qubit gate that is constructed based on values b, b + 1 and a. This is the first operation of gate U.

    a - integer value representing the index of qubit a
    b - integer value representing the index of qubit b



In [3]:
def U_ab(a, b):
    
    U = np.eye(8, dtype=complex)
    
    # Compute coefficients as illustrated in the paper
    alpha_101 = np.sqrt(comb(a - 1, b, exact=True))
    alpha_010 = np.sqrt(comb(a - 1, b + 1, exact=True))
    beta_010 = np.sqrt(comb(a, b + 1, exact=True))
    
    x = alpha_010 / beta_010
    y = alpha_101 / beta_010
    
    # Performing the superposition swap as presented in the paper.
    i_rot, j_rot = (2, 5)
    U[i_rot, i_rot] = x
    U[i_rot, j_rot] = y
    U[j_rot, i_rot] = y
    U[j_rot, j_rot] = -x
    
    return U

#### The second part of the $U$ : $U_{f}$
We are Defining the function that is creating the gate U_f. This is part of the gate U from the paper and represents a three qubit gate that is constructed based on values 0, a-1 and a. This is the second operation of gate U.

    a - integer value representing the index of qubit a

In [4]:
def U_final(a):
    
    # Compute coefficients as illustrated in the paper
    alpha_001 = 1
    alpha_100 = sqrt((a-1))
    beta_100 = sqrt(a)
    
    x = alpha_100 / beta_100
    y = alpha_001 / beta_100
    
    
    # Performing the superposition swap as presented in the paper.
    U = np.eye(8, dtype=complex)
    i_rot, j_rot = (1, 4)
    U[i_rot, i_rot] = x
    U[i_rot, j_rot] = y
    U[j_rot, i_rot] = -y
    U[j_rot, j_rot] = x
    
    return U

### Defining the quantum functions that are necessary for constructing the algorithm

#### Constructing a general function for applying an unitary matrix on three specific qubits in a circuit
This function is going to be used for applying the $U_{ab}$ and $U_f$ gates over the specific qubits in the circuit.

    matrix - input matrix that needs to be constructed before 
    q - QArray
    p1, p2, p3 - the qubits on which the gate given my matrix needs to be applied

In [5]:
@qfunc(generative=True)
def apply_matrix(matrix: CArray[CArray[CReal]], q: QArray[QBit], p1: CInt, p2: CInt, p3: CInt):
    
    ## constructing a temporary QArray of qubits representing the specific qubits in q given by p1, p2, p3
    qubits = []
    temp_array = QArray()
    for k in range(q.len):
        qubits.append(QBit(f'q{k}'))
    bind(q, qubits)
    bind([qubits[p1], qubits[p2], qubits[p3]], temp_array)
    
    
    ## aplying the matrix using the unitary built-in function
    unitary(matrix, temp_array)
        
    ## constructing back the original QArray
    bind(temp_array, [qubits[p1], qubits[p2], qubits[p3]])
    bind(qubits, q)

#### Constructing a general multi-CNOT gate that is going to be needed in tha last part of the algorithm
This function has the role of creating and applying a multi-controlled NOT operation given a set of qubit indeces. The controls list contains the list of the qubits that needs to be the ocntroll qubits and the last element in the list represents the target element.
  
    q - QArray
    controls - list of integers containing the indexes of qubits that need to be the control qubits (controls[0:-1]) and the index of the target qubit (controls[-1])

In [6]:
@qfunc(generative=True)
def apply_MultiCNOT(q: QArray[QBit], controls: CArray[CInt]):
    
    ## constructing a temporary QArray of qubits representing the specific qubits in q given by the indexes in controls
    qubits = []
    temp_array = QArray()
    for qb in range(q.len):
        qubits.append(QBit(f'q{qb}'))
    bind(q, qubits)
    bind([qubits[i] for i in controls], temp_array)    
    
    ## applying the CONTROL operation
    control(temp_array[0:len(controls)-1], lambda:X(temp_array[len(controls)-1]))
    
    
    ## constructing back the original QArray
    bind(temp_array, [qubits[i] for i in controls])
    bind(qubits, q)

#### Constructing the wrapper for the $U_{ab}$ gate that is going to be applied in the circuit
This function is a wrapper of the function U_ab and has the purpose of applying the gate for as many times as needed as it is ilustrated in the paper. U_ab needs to be applied for every qubit b from 1 to a-2.
 
    a - index of the current qubit a
    q - QArray

In [7]:
@qfunc(generative=True)
def apply_Uab(a: CReal, q: QArray[QBit]):
    
    for b in range(a-1):
        Uab = U_ab(a+1, b+1)
        apply_matrix(Uab, q, b, b+1, a)

#### Constructing the wrapper for the $U_{f}$ gate that is going to be applied in the circuit
This function is a wrapper of the function U_f and has the purpose of applying the gate 1 time as stated in paper. The gate is going to be applied on qubits 0, a-1 and a.

    a - index of the current qubit a
    q - QArray

In [8]:
@qfunc(generative=True)
def apply_Uf(a: CReal, q: QArray[QBit]):
    
    Uf = U_final(a+1)
    apply_matrix(Uf, q, 0, a-1, a) 

#### Constructing the wrapper for the whole algorithm
This function contains the core logic of the compression algorithm as presented in the paper. It takes as input a QArray representing the qubits to which the algorithm will be applied.

In [9]:
@qfunc(generative=True)
def symmetric_compression_algorithm(q_array: QArray[QBit]):
    
    ## Implementing the V gate from the paper
    control(q_array[1], lambda: X(q_array[0]))
    control(q_array[0], lambda: H(q_array[1]))
    
    
    ## Applying the U gates as in paper. This sequence will construct the |C>_k bases for the symmetric states.
    for i in range(2, q_array.len):
        apply_Uab(i, q_array)
        apply_Uf(i, q_array)
        
        # The last CNOT is the personal addition because the description in the paper is not quite exact on the 
        # implentation part. They are just describing what the functions should do.
        control(q_array[i], lambda: X(q_array[i-1]))
        
    
    ## Applying the last sequence described in paper for converting the |C>_k into |B>_k. Implementing the indications
    ## from the paper.
    for k in range(2, q_array.len):
        index = 0
        indeces = []
        for i in range(len(bin(k+1)[2:])):
            if bin(k+1)[2:][i] == '1':
                
                control(q_array[k], lambda: X(q_array[len(bin(k+1)[2:])-i-1]))
                
                indeces.append(len(bin(k+1)[2:])-i-1)
                index += 1
        indeces = indeces[::-1]
        indeces.append(k)
        apply_MultiCNOT(q_array, indeces) 

### Testing the algorithm

#### Test 1 - Compressing a quantum state of 5 identically qubits in $\left|+\right\rangle$

In [10]:
@qfunc(generative=True)
def main(x:Output[QArray]):
    
    allocate(5, x)
    hadamard_transform(x)
    symmetric_compression_algorithm(x)
    
model1 = create_model(main)
quantum_program1 = synthesize(model1)
job1 = execute(quantum_program1)
results1 = job1.result()[0].value.parsed_counts
results1
    

    
    
    
    
    

[{'x': [0, 1, 0, 0, 0]}: 650,
 {'x': [1, 1, 0, 0, 0]}: 639,
 {'x': [1, 0, 0, 0, 0]}: 322,
 {'x': [0, 0, 1, 0, 0]}: 320,
 {'x': [0, 0, 0, 0, 0]}: 59,
 {'x': [1, 0, 1, 0, 0]}: 58]

As we can see, the compression of the initial state worked, as the last two qubits are no longer used, being in state $\left|0\right\rangle$ for all of the possible states. This proves that the reduction worked, thus one being able to represent a 5 qubit state using just 3 qubits.

#### Test 2 - Compressing a quantum state of 3 qubits prepared in $W_3$ state

In [ ]:
# @qfunc(generative=True)
# def main(x:Output[QArray]):
    
#     amplitude = [0, 1/3, 1/3, 0, 1/3, 0, 0, 0]
#     prepare_amplitudes(amplitude, 0, x)
#     symmetric_compression_algorithm(x)
    
# model2 = create_model(main)
# quantum_program2 = synthesize(model2)
# job2 = execute(quantum_program2)
# results2 = job2.result()[0].value.parsed_counts
# results2

[{'x': [1, 0, 0]}: 2048]

As we can see, the compression worked, allowing us to represent the $W_3$ state using just one qubit. This opens up the possibility of using the compression algorithm in an entanglement distribution scheme, as the single qubit can be teleported using the standard quantum teleportation protocol. Also, we can see that the algorithm works for symmetric states, and not just identical ones.

#### Test 3 - Compressing a non symmetric quantum state

In [ ]:
# @qfunc(generative=True)
# def main(x:Output[QArray]):
    
#     allocate(3, x)
#     hadamard_transform(x)
#     Y(x[0])
#     symmetric_compression_algorithm(x)
    
# model3 = create_model(main)
# quantum_program3 = synthesize(model3)
# job3 = execute(quantum_program3)
# results3 = job3.result()[0].value.parsed_counts
# results3

[{'x': [1, 1, 1]}: 540,
 {'x': [0, 1, 1]}: 528,
 {'x': [0, 0, 0]}: 263,
 {'x': [1, 1, 0]}: 238,
 {'x': [1, 0, 1]}: 162,
 {'x': [0, 0, 1]}: 144,
 {'x': [0, 1, 0]}: 88,
 {'x': [1, 0, 0]}: 85]

We can see that in the case of a non-symmetric initial state, the algorithm does not perform any compression, thus confirming the hypothesis.

In [11]:
qmod = create_model(main)
write_qmod(qmod, "quantum_compression_algorithm_for_symmetric_states")